In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
pd.set_option("max_colwidth", 500)

# import sys   
# sys.setrecursionlimit(25000)

from pandarallel import pandarallel

In [2]:
headers={
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
#     'referer': 'https://www.cfr.org/defense-and-security/wars-and-conflict',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest'
}
    
    
def crawl_urls(offset, tags):
    
    if tags == 'human_rights':
        url = 'https://www.cfr.org/views/ajax?_wrapper_format=drupal_ajax&view_name=topic_filters_cards&view_display_id=filters_block&view_args=343&view_path=%2Ftaxonomy%2Fterm%2F343&view_base_path=&view_dom_id=a89c38bd6bcabb1fe44e62dd3338a45a6c9b7dbf3d2b14f13ec263040ac929a1&pager_element=0&pager_query_method=GET&_wrapper_format=html&topics=All&regions=All&type=All&'+\
        'page=%s'%offset + '&_drupal_ajax=1&ajax_page_state%5Btheme%5D=cfr_theme&ajax_page_state%5Btheme_token%5D=&ajax_page_state%5Blibraries%5D=cfr_chartbeat%2Fcfr-chartbeat-body%2Ccfr_chartbeat%2Fcfr-chartbeat-header%2Ccfr_homepage_sections%2Fadvanced_autocomplete%2Ccfr_sailthru%2Fjavascript_api_library%2Ccfr_theme%2Falert%2Ccfr_theme%2Fcard-group%2Ccfr_theme%2Ffacebook-pixel%2Ccfr_theme%2Fglobal%2Ccfr_theme%2Fheader%2Ccfr_theme%2Fheader-contextual%2Ccore%2Fdrupal.autocomplete%2Ccore%2Fhtml5shiv%2Cdatalayer%2Fbehaviors%2Cdatalayer%2Fhelper%2Cdd_datalayer_tools%2Famplitude%2Cdd_datalayer_tools%2FcustomDimensionUserCategory%2Cdd_datalayer_tools%2FdatalayerItems%2Cdd_datalayer_tools%2FpodcastFinish%2Cdd_datalayer_tools%2FpodcastStart%2Cdd_datalayer_tools%2FvideoFinish%2Cdd_datalayer_tools%2FvideoStart%2Ceu_cookie_compliance%2Feu_cookie_compliance_default%2Csearch_autocomplete%2Ftheme.minimal.css%2Csystem%2Fbase%2Cviews%2Fviews.module%2Cviews_infinite_scroll%2Fviews-infinite-scroll'

        headers['Referer'] =  'https://www.cfr.org/human-rights'
    
    if tags == 'wars_and_conflict':
        url= 'https://www.cfr.org/views/ajax?_wrapper_format=drupal_ajax&view_name=topic_filters_cards&view_display_id=filters_block&view_args=359&view_path=%2Ftaxonomy%2Fterm%2F359&view_base_path=&view_dom_id=29ba1b9157ac24788f564e6d887188223d41337fcc8e5a6e04a38fd141111fac&pager_element=0&pager_query_method=GET&_wrapper_format=html&topics=All&regions=All&type=All&'+\
        'page=%s'%offset +  '&_drupal_ajax=1&ajax_page_state%5Btheme%5D=cfr_theme&ajax_page_state%5Btheme_token%5D=&ajax_page_state%5Blibraries%5D=cfr_chartbeat%2Fcfr-chartbeat-body%2Ccfr_chartbeat%2Fcfr-chartbeat-header%2Ccfr_homepage_sections%2Fadvanced_autocomplete%2Ccfr_sailthru%2Fjavascript_api_library%2Ccfr_theme%2Falert%2Ccfr_theme%2Fcard-group%2Ccfr_theme%2Ffacebook-pixel%2Ccfr_theme%2Fglobal%2Ccfr_theme%2Fheader%2Ccfr_theme%2Fheader-contextual%2Ccore%2Fdrupal.autocomplete%2Ccore%2Fhtml5shiv%2Cdatalayer%2Fbehaviors%2Cdatalayer%2Fhelper%2Cdd_datalayer_tools%2Famplitude%2Cdd_datalayer_tools%2FcustomDimensionUserCategory%2Cdd_datalayer_tools%2FdatalayerItems%2Cdd_datalayer_tools%2FpodcastFinish%2Cdd_datalayer_tools%2FpodcastStart%2Cdd_datalayer_tools%2FvideoFinish%2Cdd_datalayer_tools%2FvideoStart%2Ceu_cookie_compliance%2Feu_cookie_compliance_default%2Csearch_autocomplete%2Ftheme.minimal.css%2Csystem%2Fbase%2Cviews%2Fviews.module%2Cviews_infinite_scroll%2Fviews-infinite-scroll'
        
        headers['Referer'] = 'https://www.cfr.org/defense-and-security/wars-and-conflict'
        
    if tags == 'territorial_disputes':  
        
        url= 'https://www.cfr.org/views/ajax?_wrapper_format=drupal_ajax&view_name=topic_filters_cards&view_display_id=filters_block&view_args=356&view_path=%2Ftaxonomy%2Fterm%2F356&view_base_path=&view_dom_id=5ea33c8cd3608759dce20248a26f39eb7cfb5624b7e69708003d92502d420e13&pager_element=0&pager_query_method=GET&_wrapper_format=html&topics=All&regions=All&type=All&'+\
        'page=%s'%offset +  '&_drupal_ajax=1&ajax_page_state%5Btheme%5D=cfr_theme&ajax_page_state%5Btheme_token%5D=&ajax_page_state%5Blibraries%5D=cfr_chartbeat%2Fcfr-chartbeat-body%2Ccfr_chartbeat%2Fcfr-chartbeat-header%2Ccfr_homepage_sections%2Fadvanced_autocomplete%2Ccfr_sailthru%2Fjavascript_api_library%2Ccfr_theme%2Falert%2Ccfr_theme%2Ffacebook-pixel%2Ccfr_theme%2Fglobal%2Ccfr_theme%2Fheader%2Ccfr_theme%2Fheader-contextual%2Ccore%2Fdrupal.autocomplete%2Ccore%2Fhtml5shiv%2Cdatalayer%2Fbehaviors%2Cdatalayer%2Fhelper%2Cdd_datalayer_tools%2Famplitude%2Cdd_datalayer_tools%2FcustomDimensionUserCategory%2Cdd_datalayer_tools%2FdatalayerItems%2Cdd_datalayer_tools%2FpodcastFinish%2Cdd_datalayer_tools%2FpodcastStart%2Cdd_datalayer_tools%2FvideoFinish%2Cdd_datalayer_tools%2FvideoStart%2Ceu_cookie_compliance%2Feu_cookie_compliance_default%2Csearch_autocomplete%2Ftheme.minimal.css%2Csystem%2Fbase%2Cviews%2Fviews.module%2Cviews_infinite_scroll%2Fviews-infinite-scroll'
        
        headers['Referer'] = 'https://www.cfr.org/defense-and-security/territorial-disputes'

    if tags == 'extremism':  

        url= 'https://www.cfr.org/views/ajax?_wrapper_format=drupal_ajax&view_name=topic_filters_cards&view_display_id=filters_block&view_args=428&view_path=%2Ftaxonomy%2Fterm%2F428&view_base_path=&view_dom_id=d1218686666fee2e99ff88a01defdf38a3dd49f79a8cfe11990cfda72e8cf3c3&pager_element=0&pager_query_method=GET&_wrapper_format=html&topics=All&regions=All&type=All&'+\
        'page=%s'%offset +  '&_drupal_ajax=1&ajax_page_state%5Btheme%5D=cfr_theme&ajax_page_state%5Btheme_token%5D=&ajax_page_state%5Blibraries%5D=cfr_chartbeat%2Fcfr-chartbeat-body%2Ccfr_chartbeat%2Fcfr-chartbeat-header%2Ccfr_homepage_sections%2Fadvanced_autocomplete%2Ccfr_sailthru%2Fjavascript_api_library%2Ccfr_theme%2Falert%2Ccfr_theme%2Fcard-group%2Ccfr_theme%2Ffacebook-pixel%2Ccfr_theme%2Fglobal%2Ccfr_theme%2Fheader%2Ccfr_theme%2Fheader-contextual%2Ccore%2Fdrupal.autocomplete%2Ccore%2Fhtml5shiv%2Cdatalayer%2Fbehaviors%2Cdatalayer%2Fhelper%2Cdd_datalayer_tools%2Famplitude%2Cdd_datalayer_tools%2FcustomDimensionUserCategory%2Cdd_datalayer_tools%2FdatalayerItems%2Cdd_datalayer_tools%2FpodcastFinish%2Cdd_datalayer_tools%2FpodcastStart%2Cdd_datalayer_tools%2FvideoFinish%2Cdd_datalayer_tools%2FvideoStart%2Ceu_cookie_compliance%2Feu_cookie_compliance_default%2Csearch_autocomplete%2Ftheme.minimal.css%2Csystem%2Fbase%2Cviews%2Fviews.module%2Cviews_infinite_scroll%2Fviews-infinite-scroll'
        
        headers['Referer'] = 'https://www.cfr.org/social-issues/radicalization-and-extremism'
        
    
    if tags == 'defense':  
        url= 'https://www.cfr.org/views/ajax?_wrapper_format=drupal_ajax&view_name=topic_filters_cards&view_display_id=filters_block&view_args=338&view_path=%2Ftaxonomy%2Fterm%2F338&view_base_path=&view_dom_id=530b9fce4af47131728c1365b973acf92a242c863a935110c7d64d8a8dffa503&pager_element=0&pager_query_method=GET&_wrapper_format=html&topics=All&regions=All&type=All&'+\
        'page=%s'%offset + '&_drupal_ajax=1&ajax_page_state%5Btheme%5D=cfr_theme&ajax_page_state%5Btheme_token%5D=&ajax_page_state%5Blibraries%5D=cfr_chartbeat%2Fcfr-chartbeat-body%2Ccfr_chartbeat%2Fcfr-chartbeat-header%2Ccfr_homepage_sections%2Fadvanced_autocomplete%2Ccfr_sailthru%2Fjavascript_api_library%2Ccfr_theme%2Falert%2Ccfr_theme%2Fcard-group%2Ccfr_theme%2Ffacebook-pixel%2Ccfr_theme%2Fglobal%2Ccfr_theme%2Fheader%2Ccfr_theme%2Fheader-contextual%2Ccore%2Fdrupal.autocomplete%2Ccore%2Fhtml5shiv%2Cdatalayer%2Fbehaviors%2Cdatalayer%2Fhelper%2Cdd_datalayer_tools%2Famplitude%2Cdd_datalayer_tools%2FcustomDimensionUserCategory%2Cdd_datalayer_tools%2FdatalayerItems%2Cdd_datalayer_tools%2FpodcastFinish%2Cdd_datalayer_tools%2FpodcastStart%2Cdd_datalayer_tools%2FvideoFinish%2Cdd_datalayer_tools%2FvideoStart%2Ceu_cookie_compliance%2Feu_cookie_compliance_default%2Csearch_autocomplete%2Ftheme.minimal.css%2Csystem%2Fbase%2Cviews%2Fviews.module%2Cviews_infinite_scroll%2Fviews-infinite-scroll'

        headers['Referer'] = 'https://www.cfr.org/defense-and-security'


    if tags == 'diplomacy':  

        url= 'https://www.cfr.org/views/ajax?_wrapper_format=drupal_ajax&view_name=topic_filters_cards&view_display_id=filters_block&view_args=339&view_path=%2Ftaxonomy%2Fterm%2F339&view_base_path=&view_dom_id=7bf2165a882bc0375bf7a5ee7740c8c7aa0ac30e339e959ae7384a64f08e33f1&pager_element=0&pager_query_method=GET&_wrapper_format=html&topics=All&regions=All&type=All&'+\
        'page=%s'%offset + '&_drupal_ajax=1&ajax_page_state%5Btheme%5D=cfr_theme&ajax_page_state%5Btheme_token%5D=&ajax_page_state%5Blibraries%5D=cfr_chartbeat%2Fcfr-chartbeat-body%2Ccfr_chartbeat%2Fcfr-chartbeat-header%2Ccfr_homepage_sections%2Fadvanced_autocomplete%2Ccfr_sailthru%2Fjavascript_api_library%2Ccfr_theme%2Falert%2Ccfr_theme%2Fcard-group%2Ccfr_theme%2Ffacebook-pixel%2Ccfr_theme%2Fglobal%2Ccfr_theme%2Fheader%2Ccfr_theme%2Fheader-contextual%2Ccore%2Fdrupal.autocomplete%2Ccore%2Fhtml5shiv%2Cdatalayer%2Fbehaviors%2Cdatalayer%2Fhelper%2Cdd_datalayer_tools%2Famplitude%2Cdd_datalayer_tools%2FcustomDimensionUserCategory%2Cdd_datalayer_tools%2FdatalayerItems%2Cdd_datalayer_tools%2FpodcastFinish%2Cdd_datalayer_tools%2FpodcastStart%2Cdd_datalayer_tools%2FvideoFinish%2Cdd_datalayer_tools%2FvideoStart%2Ceu_cookie_compliance%2Feu_cookie_compliance_default%2Csearch_autocomplete%2Ftheme.minimal.css%2Csystem%2Fbase%2Cviews%2Fviews.module%2Cviews_infinite_scroll%2Fviews-infinite-scroll'

        headers['Referer'] = 'https://www.cfr.org/diplomacy-and-international-institutions'

    if tags == 'politics':  

        url= 'https://www.cfr.org/views/ajax?_wrapper_format=drupal_ajax&view_name=topic_filters_cards&view_display_id=filters_block&view_args=344&view_path=%2Ftaxonomy%2Fterm%2F344&view_base_path=&view_dom_id=eef78b57432bc6590594e3b25e1b68883e98831d7427717977cc75549fbb8f6e&pager_element=0&pager_query_method=GET&_wrapper_format=html&topics=All&regions=All&type=All&'+\
        'page=%s'%offset + '&_drupal_ajax=1&ajax_page_state%5Btheme%5D=cfr_theme&ajax_page_state%5Btheme_token%5D=&ajax_page_state%5Blibraries%5D=cfr_chartbeat%2Fcfr-chartbeat-body%2Ccfr_chartbeat%2Fcfr-chartbeat-header%2Ccfr_homepage_sections%2Fadvanced_autocomplete%2Ccfr_sailthru%2Fjavascript_api_library%2Ccfr_theme%2Falert%2Ccfr_theme%2Fcard-group%2Ccfr_theme%2Ffacebook-pixel%2Ccfr_theme%2Fglobal%2Ccfr_theme%2Fheader%2Ccfr_theme%2Fheader-contextual%2Ccore%2Fdrupal.autocomplete%2Ccore%2Fhtml5shiv%2Cdatalayer%2Fbehaviors%2Cdatalayer%2Fhelper%2Cdd_datalayer_tools%2Famplitude%2Cdd_datalayer_tools%2FcustomDimensionUserCategory%2Cdd_datalayer_tools%2FdatalayerItems%2Cdd_datalayer_tools%2FpodcastFinish%2Cdd_datalayer_tools%2FpodcastStart%2Cdd_datalayer_tools%2FvideoFinish%2Cdd_datalayer_tools%2FvideoStart%2Ceu_cookie_compliance%2Feu_cookie_compliance_default%2Csearch_autocomplete%2Ftheme.minimal.css%2Csystem%2Fbase%2Cviews%2Fviews.module%2Cviews_infinite_scroll%2Fviews-infinite-scroll'

        headers['Referer'] = 'https://www.cfr.org/politics-and-government'
        
    if tags == 'social':  

        url= 'https://www.cfr.org/views/ajax?_wrapper_format=drupal_ajax&view_name=topic_filters_cards&view_display_id=filters_block&view_args=345&view_path=%2Ftaxonomy%2Fterm%2F345&view_base_path=&view_dom_id=a0dbe6e0741b05f5f8262b4b8fcab8318f7f128eae4bb39912640f203ec71379&pager_element=0&pager_query_method=GET&_wrapper_format=html&topics=All&regions=All&type=All&'+\
        'page=%s'%offset + '&_drupal_ajax=1&ajax_page_state%5Btheme%5D=cfr_theme&ajax_page_state%5Btheme_token%5D=&ajax_page_state%5Blibraries%5D=cfr_chartbeat%2Fcfr-chartbeat-body%2Ccfr_chartbeat%2Fcfr-chartbeat-header%2Ccfr_homepage_sections%2Fadvanced_autocomplete%2Ccfr_sailthru%2Fjavascript_api_library%2Ccfr_theme%2Falert%2Ccfr_theme%2Fcard-group%2Ccfr_theme%2Ffacebook-pixel%2Ccfr_theme%2Fglobal%2Ccfr_theme%2Fheader%2Ccfr_theme%2Fheader-contextual%2Ccore%2Fdrupal.autocomplete%2Ccore%2Fhtml5shiv%2Cdatalayer%2Fbehaviors%2Cdatalayer%2Fhelper%2Cdd_datalayer_tools%2Famplitude%2Cdd_datalayer_tools%2FcustomDimensionUserCategory%2Cdd_datalayer_tools%2FdatalayerItems%2Cdd_datalayer_tools%2FpodcastFinish%2Cdd_datalayer_tools%2FpodcastStart%2Cdd_datalayer_tools%2FvideoFinish%2Cdd_datalayer_tools%2FvideoStart%2Ceu_cookie_compliance%2Feu_cookie_compliance_default%2Csearch_autocomplete%2Ftheme.minimal.css%2Csystem%2Fbase%2Cviews%2Fviews.module%2Cviews_infinite_scroll%2Fviews-infinite-scroll'

        headers['Referer'] = 'https://www.cfr.org/social-issues'


    res=requests.get(url, headers=headers)
    # time.sleep(2)

    if res.status_code==200:
        js = res.json()
        soup = BeautifulSoup(js[1]['data'], 'html.parser')
        rows = soup.find_all('div', class_="card-article-large__content")
        rows = [str(i) for i in rows]
        return rows
    else:
        print('Error: %s'%offset)
        return

In [26]:
# crawl_urls(110,'human_rights')
# crawl_urls(80,'wars_and_conflict')
# crawl_urls(2,'territorial_disputes')
# crawl_urls(8,'extremism')
# crawl_urls(8,'extremism')

# crawl_urls(400,'defense')
# crawl_urls(400,'diplomacy')
# crawl_urls(450,'politics')
crawl_urls(120,'social')

[]

In [27]:
df0 = [[i, 'defense'] for i in range(1, 400)]
df1 = [ [i, 'diplomacy'] for i in range(1, 400)]
df2 = [ [i, 'politics' ] for i in range(1, 450)]
df3 = [ [i, 'social' ] for i in range(1, 120)]

df = pd.DataFrame(df0+ df1 + df2 + df3, columns= ['offset', 'tags'])
df

,offset,tags
0,1,defense
1,2,defense
2,3,defense
3,4,defense
4,5,defense
...,...,...
1361,115,social
1362,116,social
1363,117,social
1364,118,social


In [28]:
pandarallel.initialize(nb_workers= 32, progress_bar = True)
df[['rows']] = df.parallel_apply(lambda x: crawl_urls(x[0],x[1]), axis=1).values

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [36]:
df = df.explode('rows').reset_index(drop=True)
df = df.iloc[:,1:]
df = df[~df.rows.isnull()].reset_index(drop=True)
df

,tags,rows,title,date,link
0,defense,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/defense-and-security/cybersecurity"">\n Cybersecurity\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/blog/cyber-week-review-march-26-2021 "">\n<div class=""card-article-large__title"">\n Cyber Week in Review: March 26, 2021\n </div>\n<div class=""card-article-large__image"">\n<div class=""card-article-large__im...","Cyber Week in Review: March 26, 2021","March 26, 2021",https://www.cfr.org/blog/cyber-week-review-march-26-2021
1,defense,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/europe-and-eurasia"">\n Europe and Eurasia\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/in-brief/armenias-postwar-crisis-what-know "">\n<div class=""card-article-large__title"">\n Armenia’s Postwar Crisis: What to Know\n </div>\n<div class=""card-article-large__image"">\n<div class=""card-article-large__image...",Armenia’s Postwar Crisis: What to Know,"March 25, 2021",https://www.cfr.org/in-brief/armenias-postwar-crisis-what-know
2,defense,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/defense-and-security/terrorism-and-counterterrorism"">\n Terrorism and Counterterrorism\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/article/public-should-be-warned-when-rampage-underway "">\n<div class=""card-article-large__title"">\n The Public Should Be Warned When a Rampage Is Underway\n </div>\n<div c...",The Public Should Be Warned When a Rampage Is Underway,"March 24, 2021",https://www.cfr.org/article/public-should-be-warned-when-rampage-underway
3,defense,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/diplomacy-and-international-institutions/international-law"">\n International Law\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/report/reforming-war-powers-resolution-21st-century "">\n<div class=""card-article-large__title"">\n Reforming the War Powers Resolution for the 21st Century\n </div>\n</a>\n<div c...",Reforming the War Powers Resolution for the 21st Century,"March 23, 2021",https://www.cfr.org/report/reforming-war-powers-resolution-21st-century
4,defense,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/defense-and-security/terrorism-and-counterterrorism"">\n Terrorism and Counterterrorism\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/blog/multiple-jihadi-insurgencies-cooperating-bandits-appear-be-converging-sahel "">\n<div class=""card-article-large__title"">\n Multiple Jihadi Insurgencies, Cooperating With Band...","Multiple Jihadi Insurgencies, Cooperating With Bandits, Appear to be Converging in the Sahel","March 23, 2021",https://www.cfr.org/blog/multiple-jihadi-insurgencies-cooperating-bandits-appear-be-converging-sahel
...,...,...,...,...,...
18478,social,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/asia/china"">\n China\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/blog/samsung-scandal-islamic-state-and-china-philippine-hiv-and-more "">\n<div class=""card-article-large__title"">\n Samsung Scandal, Islamic State and China, Philippine HIV, and More\n </div>\n<div class=""card-article-large__image"">\n<div...","Samsung Scandal, Islamic State and China, Philippine HIV, and More","March 3, 2017",https://www.cfr.org/blog/samsung-scandal-islamic-state-and-china-philippine-hiv-and-more
18479,social,"<div class=""card-article-large__content"">\n<div class=""card-article-large__

In [30]:
def extract_row(row):
    soup = BeautifulSoup(row, 'html.parser')
    try:
        a = soup.find_all('a', class_ ="card-article-large__link")[0]
        link = a['href']
        link = re.sub("^\s+|\s+$", "",link, flags=re.UNICODE) if link else ''
        link = 'https://www.cfr.org'+ link if link.startswith('/') else link
        
        title = re.sub("^\s+|\s+$", "", a.div.text, flags=re.UNICODE) if a.div.text else ''
        
        date = soup.find_all('span', class_ ="card-article-large__date")[0].text
        date = re.sub("^\s+|\s+$", "", date, flags=re.UNICODE) if date else ''
        return pd.Series([title, date, link])
    
    except:
        return pd.Series(['error']*4)

In [37]:
pandarallel.initialize(nb_workers= 16)
df[['title','date','link']] = df['rows'].parallel_apply(extract_row).values

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [38]:
df = df[~df.link.duplicated()].reset_index(drop=True)
df

,tags,rows,title,date,link
0,defense,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/defense-and-security/cybersecurity"">\n Cybersecurity\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/blog/cyber-week-review-march-26-2021 "">\n<div class=""card-article-large__title"">\n Cyber Week in Review: March 26, 2021\n </div>\n<div class=""card-article-large__image"">\n<div class=""card-article-large__im...","Cyber Week in Review: March 26, 2021","March 26, 2021",https://www.cfr.org/blog/cyber-week-review-march-26-2021
1,defense,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/europe-and-eurasia"">\n Europe and Eurasia\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/in-brief/armenias-postwar-crisis-what-know "">\n<div class=""card-article-large__title"">\n Armenia’s Postwar Crisis: What to Know\n </div>\n<div class=""card-article-large__image"">\n<div class=""card-article-large__image...",Armenia’s Postwar Crisis: What to Know,"March 25, 2021",https://www.cfr.org/in-brief/armenias-postwar-crisis-what-know
2,defense,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/defense-and-security/terrorism-and-counterterrorism"">\n Terrorism and Counterterrorism\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/article/public-should-be-warned-when-rampage-underway "">\n<div class=""card-article-large__title"">\n The Public Should Be Warned When a Rampage Is Underway\n </div>\n<div c...",The Public Should Be Warned When a Rampage Is Underway,"March 24, 2021",https://www.cfr.org/article/public-should-be-warned-when-rampage-underway
3,defense,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/diplomacy-and-international-institutions/international-law"">\n International Law\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/report/reforming-war-powers-resolution-21st-century "">\n<div class=""card-article-large__title"">\n Reforming the War Powers Resolution for the 21st Century\n </div>\n</a>\n<div c...",Reforming the War Powers Resolution for the 21st Century,"March 23, 2021",https://www.cfr.org/report/reforming-war-powers-resolution-21st-century
4,defense,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/defense-and-security/terrorism-and-counterterrorism"">\n Terrorism and Counterterrorism\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/blog/multiple-jihadi-insurgencies-cooperating-bandits-appear-be-converging-sahel "">\n<div class=""card-article-large__title"">\n Multiple Jihadi Insurgencies, Cooperating With Band...","Multiple Jihadi Insurgencies, Cooperating With Bandits, Appear to be Converging in the Sahel","March 23, 2021",https://www.cfr.org/blog/multiple-jihadi-insurgencies-cooperating-bandits-appear-be-converging-sahel
...,...,...,...,...,...
14411,social,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/middle-east-and-north-africa"">\n Middle East and North Africa\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/event/fifth-annual-back-school-event "">\n<div class=""card-article-large__title"">\n Fifth Annual Back-to-School Event\n </div>\n<div class=""card-article-large__image"">\n<span class=""card-article-la...",Fifth Annual Back-to-School Event,"December 31, 1969",https://www.cfr.org/event/fifth-annual-back-school-event
14412,social,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" 

In [54]:
# df_others = pd.read_csv('cfr_final.csv', header=[0])
# df_others

In [55]:
# df_others.shape

In [57]:
df = df[['tags','title','date','link','text']]
df

,tags,title,date,link,text
0,defense,"Cyber Week in Review: March 26, 2021","March 26, 2021",https://www.cfr.org/blog/cyber-week-review-march-26-2021,NaN
1,defense,Armenia’s Postwar Crisis: What to Know,"March 25, 2021",https://www.cfr.org/in-brief/armenias-postwar-crisis-what-know,NaN
2,defense,The Public Should Be Warned When a Rampage Is Underway,"March 24, 2021",https://www.cfr.org/article/public-should-be-warned-when-rampage-underway,NaN
3,defense,Reforming the War Powers Resolution for the 21st Century,"March 23, 2021",https://www.cfr.org/report/reforming-war-powers-resolution-21st-century,NaN
4,defense,"Multiple Jihadi Insurgencies, Cooperating With Bandits, Appear to be Converging in the Sahel","March 23, 2021",https://www.cfr.org/blog/multiple-jihadi-insurgencies-cooperating-bandits-appear-be-converging-sahel,NaN
...,...,...,...,...,...
14927,extremism,UK Keynote: A New Approach to Counter-Radicalization,"April 1, 2011",https://www.cfr.org/event/uk-keynote-new-approach-counter-radicalization,"\nThis session was part of the symposium, UK and U.S. Approaches in Countering Radicalization: Intelligence, Communities, and the Internet, which was cosponsored with Georgetown University’s Center for Peace and Security Studies and King’s College London’s International Centre for the Study of Radicalisation. This event was made possible by Georgetown University's George T. Kalaris Intelligence Studies Fund and the generous support of longtime CFR member Rita E. Hauser. Additionally, this e..."
14928,extremism,Panel Two: Intelligence and Counter-Radicalization,"April 1, 2011",https://www.cfr.org/event/panel-two-intelligence-and-counter-radicalization,"This session was part of the symposium, UK and U.S. Approaches in Countering Radicalization: Intelligence, Communities, and the Internet, which was cosponsored with Georgetown University’s Center for Peace and Security Studies and King’s College London’s International Centre for the Study of Radicalisation. This event was made possible by Georgetown University's George T. Kalaris Intelligence Studies Fund and the generous support of longtime CFR member Rita E. Hauser. Additionally, this even..."
14929,extremism,U.S. Keynote: Community Partnerships to Counter Violent Extremism,"April 1, 2011",https://www.cfr.org/event/us-keynote-community-partnerships-counter-violent-extremism,"This session was part of the symposium, UK and U.S. Approaches in Countering Radicalization: Intelligence, Communities, and the Internet, which was cosponsored with Georgetown University’s Center for Peace and Security Studies and King’s College London’s International Centre for the Study of Radicalisation. This event was made possible by Georgetown University's George T. Kalaris Intelligence Studies Fund and the generous support of longtime CFR member Rita E. Hauser. Additionally, this even..."
14930,extremism,Radicalization and U.S. Muslims,"March 11, 2011",https://www.cfr.org/interview/radicalization-and-us-muslims,"More on:\n\nUnited States\n\n\nRadicalization and Extremism\n\nAmid the debate over the threat posed by homegrown Islamic terrorism in the United States, a central aspect is the nature of the Islamic community’s cooperation in counterterrorism efforts. Mark Fallon, a thirty-year veteran of federal law enforcement and counterintelligence, says the Muslim community has provided a ""significant level of cooperation"" in combating terrorism, and worries that the rhetoric from some critics risks al..."


In [58]:
# df = df[~df.link.isin(df_others.link)].reset_index(drop=True)
# df.to_csv('cfr_raw_more.csv', index=False)
df.to_csv('cfr_all.csv', index=False)

In [3]:
# df = pd.read_csv('cfr_raw.csv', header=[0])
# df.sort_values('date')

,tags,rows,title,date,link
2610,wars_and_conflict,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/americas/united-states"">\n United States\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/interview/capital-interview-general-william-lord-cyberspace-and-future-warfare "">\n<div class=""card-article-large__title"">\n The Capital Interview: General William Lord on Cyberspace and the Future of Warfare\n </div>...",The Capital Interview: General William Lord on Cyberspace and the Future of Warfare,"April 1, 2008",https://www.cfr.org/interview/capital-interview-general-william-lord-cyberspace-and-future-warfare
2792,extremism,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/europe-and-eurasia/united-kingdom"">\n United Kingdom\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/event/uk-keynote-new-approach-counter-radicalization "">\n<div class=""card-article-large__title"">\n UK Keynote: A New Approach to Counter-Radicalization\n </div>\n<div class=""card-article-large__image"">\n<s...",UK Keynote: A New Approach to Counter-Radicalization,"April 1, 2011",https://www.cfr.org/event/uk-keynote-new-approach-counter-radicalization
2788,extremism,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/europe-and-eurasia/united-kingdom"">\n United Kingdom\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/event/new-approach-counter-radicalization "">\n<div class=""card-article-large__title"">\n A New Approach to Counter-Radicalization\n </div>\n<div class=""card-article-large__image"">\n<span class=""card-article...",A New Approach to Counter-Radicalization,"April 1, 2011",https://www.cfr.org/event/new-approach-counter-radicalization
2789,extremism,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/europe-and-eurasia/united-kingdom"">\n United Kingdom\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/event/community-partnerships-counter-violent-extremism "">\n<div class=""card-article-large__title"">\n Community Partnerships to Counter Violent Extremism\n </div>\n<div class=""card-article-large__image"">\n<...",Community Partnerships to Counter Violent Extremism,"April 1, 2011",https://www.cfr.org/event/community-partnerships-counter-violent-extremism
2794,extremism,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/europe-and-eurasia/united-kingdom"">\n United Kingdom\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/event/us-keynote-community-partnerships-counter-violent-extremism "">\n<div class=""card-article-large__title"">\n U.S. Keynote: Community Partnerships to Counter Violent Extremism\n </div>\n<div class=""card-...",U.S. Keynote: Community Partnerships to Counter Violent Extremism,"April 1, 2011",https://www.cfr.org/event/us-keynote-community-partnerships-counter-violent-extremism
...,...,...,...,...,...
2172,wars_and_conflict,"<div class=""card-article-large__content"">\n<div class=""card-article-large__topic-tag"">\n<a class=""card-article-large__topic-tag-link"" href=""/sub-saharan-africa"">\n Sub-Saharan Africa\n </a>\n</div>\n<a class=""card-article-large__link"" href=""/blog/reactions-us-strike-somalia "">\n<div class=""card-article-large__title"">\n Reactions to the U.S. Strike in Somalia\n </div>\n<div class=""card-article-large__image"">\n<div class=""card-article-large__image-cover"" s...",Reactions to the U.S. Strike in Somalia,"September 9, 2014",https://www.cfr.org/blog/reactions-us-strike-somalia
2013,wars_and_conflict,"<div class=""card-article-large__content"">\n<div class

In [6]:
headers ={
'accept': '*/*',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'zh,zh-TW;q=0.9,zh-CN;q=0.8,en;q=0.7',
'origin': 'https://www.cfr.org',
'referer': 'https://www.cfr.org/',
'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
'sec-ch-ua-mobile': '?0',
'sec-fetch-dest': 'empty',
'sec-fetch-mode': 'cors',
'sec-fetch-site': 'cross-site',
'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}


def get_html(url):
    try:
        coverpage =requests.get(url, headers= headers).content   
        soup = BeautifulSoup(coverpage, 'html.parser') 
    except:
        soup = 'error'
        print('error', url)
        
    return str(soup)
        
#         html = soup.find_all('div', class_ = 'body-content')[0]
#         text = [i.text for i in html.find_all('p')]
#         text = '\n'.join(text)
        
#     except:
#         html = 'error'
#         text = 'error'
#         print('error', url)
      
#     return pd.Series([str(html), text])



from newspaper import Article, Config
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

def get_text_newspaper(url):
    try:
        a = Article(url, config= config)
        a.download()
        a.parse()
        text= a.text
    except:
        text = 'error'
        print('error', url)
        
    return text

In [7]:
df = pd.read_csv('cfr_all.csv', header=[0])
df

,tags,title,date,link,text
0,defense,"Cyber Week in Review: March 26, 2021","March 26, 2021",https://www.cfr.org/blog/cyber-week-review-march-26-2021,NaN
1,defense,Armenia’s Postwar Crisis: What to Know,"March 25, 2021",https://www.cfr.org/in-brief/armenias-postwar-crisis-what-know,NaN
2,defense,The Public Should Be Warned When a Rampage Is Underway,"March 24, 2021",https://www.cfr.org/article/public-should-be-warned-when-rampage-underway,NaN
3,defense,Reforming the War Powers Resolution for the 21st Century,"March 23, 2021",https://www.cfr.org/report/reforming-war-powers-resolution-21st-century,NaN
4,defense,"Multiple Jihadi Insurgencies, Cooperating With Bandits, Appear to be Converging in the Sahel","March 23, 2021",https://www.cfr.org/blog/multiple-jihadi-insurgencies-cooperating-bandits-appear-be-converging-sahel,NaN
...,...,...,...,...,...
14927,extremism,UK Keynote: A New Approach to Counter-Radicalization,"April 1, 2011",https://www.cfr.org/event/uk-keynote-new-approach-counter-radicalization,"\nThis session was part of the symposium, UK and U.S. Approaches in Countering Radicalization: Intelligence, Communities, and the Internet, which was cosponsored with Georgetown University’s Center for Peace and Security Studies and King’s College London’s International Centre for the Study of Radicalisation. This event was made possible by Georgetown University's George T. Kalaris Intelligence Studies Fund and the generous support of longtime CFR member Rita E. Hauser. Additionally, this e..."
14928,extremism,Panel Two: Intelligence and Counter-Radicalization,"April 1, 2011",https://www.cfr.org/event/panel-two-intelligence-and-counter-radicalization,"This session was part of the symposium, UK and U.S. Approaches in Countering Radicalization: Intelligence, Communities, and the Internet, which was cosponsored with Georgetown University’s Center for Peace and Security Studies and King’s College London’s International Centre for the Study of Radicalisation. This event was made possible by Georgetown University's George T. Kalaris Intelligence Studies Fund and the generous support of longtime CFR member Rita E. Hauser. Additionally, this even..."
14929,extremism,U.S. Keynote: Community Partnerships to Counter Violent Extremism,"April 1, 2011",https://www.cfr.org/event/us-keynote-community-partnerships-counter-violent-extremism,"This session was part of the symposium, UK and U.S. Approaches in Countering Radicalization: Intelligence, Communities, and the Internet, which was cosponsored with Georgetown University’s Center for Peace and Security Studies and King’s College London’s International Centre for the Study of Radicalisation. This event was made possible by Georgetown University's George T. Kalaris Intelligence Studies Fund and the generous support of longtime CFR member Rita E. Hauser. Additionally, this even..."
14930,extremism,Radicalization and U.S. Muslims,"March 11, 2011",https://www.cfr.org/interview/radicalization-and-us-muslims,"More on:\n\nUnited States\n\n\nRadicalization and Extremism\n\nAmid the debate over the threat posed by homegrown Islamic terrorism in the United States, a central aspect is the nature of the Islamic community’s cooperation in counterterrorism efforts. Mark Fallon, a thirty-year veteran of federal law enforcement and counterintelligence, says the Muslim community has provided a ""significant level of cooperation"" in combating terrorism, and worries that the rhetoric from some critics risks al..."


In [7]:
pandarallel.initialize(nb_workers=16)
batch_size = 16 * 8
i=resume = 512
while i<= df.shape[0]:
    time.sleep(1)
    print(i, end=',')
    df.loc[i: i+ batch_size-1, 'text']= df.loc[i: i+ batch_size-1, 'link'].parallel_apply(get_text_newspaper).values
    i += batch_size
    if (i-resume)%(batch_size) == 4:
        df.to_csv('df_temp.csv', index=False)
df.to_csv('df_temp.csv', index=False)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
512,640,768,896,error https://www.cfr.org/report/challenge-north-korea
1024,1152,error https://www.cfr.org/expert-brief/uncertainty-among-us-allies-northeast-asia
1280,1408,1536,1664,1792,1920,2048,2176,2304,error https://www.cfr.org/publication/interactive/29673
2432,2560,2688,2816,2944,3072,3200,3328,3456,3584,3712,error https://www.cfr.org/publication/interactive/11954
error https://www.cfr.org/publication/interactive/13850
3840,3968,4096,4224,4352,4480,4608,4736,4864,4992,5120,5248,5376,5504,5632,5760,5888,6016,6144,6272,6400,6528,6656,6784,6912,7040,7168,7296,7424,7552,7680,7808,7936,8064,8192,8320,8448,8576,8704,8832,8960,9088,9216,9344,9472,9600,9728,9856,9984,10112,error https://www.cfr.org/publication/interactive/26847
10240,10368,10496,10624,error https://www.cfr.org/publication/interactive/20163
10752,10880,error https://www.cfr.org/p

In [12]:
df = df[df.text!='error'].reset_index(drop=True)
df

,tags,title,date,link,text
0,defense,"Cyber Week in Review: March 26, 2021","March 26, 2021",https://www.cfr.org/blog/cyber-week-review-march-26-2021,"China Restricts Use of Tesla Vehicles by Military and State Employees\n\nFollowing a government security review, the Chinese military has announced that it will restrict the use of Tesla Inc’s vehicles by both military staff and employees of certain state-owned enterprises. The ban is in response to security concerns regarding Tesla’s ubiquitous camera and telecommunications technology, which Beijing believes could facilitate covert data extraction and surveillance. The move mirrors U.S. res..."
1,defense,Armenia’s Postwar Crisis: What to Know,"March 25, 2021",https://www.cfr.org/in-brief/armenias-postwar-crisis-what-know,"Armenian Prime Minister Nikol Pashinyan has called for early elections after weeks of protests and a standoff with his own army. What’s going on?\n\nPashinyan, Armenia’s forty-five-year-old prime minister who was elected in 2018 on the back of a so-called Velvet Revolution, is banking on early elections set for June 20 to quell a political crisis that has its roots in Armenia’s bitter defeat in a forty-four-day war with neighboring Azerbaijan last year.\n\nPashinyan himself triggered the cri..."
2,defense,The Public Should Be Warned When a Rampage Is Underway,"March 24, 2021",https://www.cfr.org/article/public-should-be-warned-when-rampage-underway,"Mass shootings and domestic terrorism are often remembered in terms of places and buildings—single symbolic locations where horror unfolded. The historic Emanuel African Methodist Episcopal Church in Charleston, South Carolina; a Walmart in El Paso, Texas; and the Tree of Life Congregation synagogue in Pittsburgh all readily come to mind.\n\nBut many assailants embark on paths of destruction across multiple crime scenes. An attack that begins as what seems to be an isolated incident at one s..."
3,defense,Reforming the War Powers Resolution for the 21st Century,"March 23, 2021",https://www.cfr.org/report/reforming-war-powers-resolution-21st-century,"Economics\n\nBetsey Stevenson, professor of public policy and economics at University of Michigan, leads a conversation on the new U.S. stimulus bill and how it will shape U.S. economic competitiveness.\n\nVirtual Event with Betsey Stevenson State and Local Conference Calls and Webinars"
4,defense,"Multiple Jihadi Insurgencies, Cooperating With Bandits, Appear to be Converging in the Sahel","March 23, 2021",https://www.cfr.org/blog/multiple-jihadi-insurgencies-cooperating-bandits-appear-be-converging-sahel,"On March 17, around one hundred assailants, traveling on motorcycles and pickup trucks, killed thirty-three soldiers and wounded an additional fourteen in an attack near Tessit in central Mali. Peacekeepers operating under the UN Multidimensional Integrated Stabilization Mission in Mali (MINUSMA) assisted the evacuation of the dead and wounded; the French-led counterinsurgency force stationed in West Africa, Operation Barkhane, helped the Malian military secure the area after the attack. Abo..."
...,...,...,...,...,...
14915,extremism,UK Keynote: A New Approach to Counter-Radicalization,"April 1, 2011",https://www.cfr.org/event/uk-keynote-new-approach-counter-radicalization,"This session was part of the symposium, UK and U.S. Approaches in Countering Radicalization: Intelligence, Communities, and the Internet, which was cosponsored with Georgetown University’s Center for Peace and Security Studies and King’s College London’s International Centre for the Study of Radicalisation. This event was made possible by Georgetown University's George T. Kalaris Intelligence Studies Fund and the generous support of longtime CFR member Rita E. Hauser. Additionally, this even..."
14916,extremism,Panel Two: Intelligence and Counter-Radicalization,"April 1, 2011",https://www.cfr.org/event/panel-two-intelligence-and-counter-radicalization,"This session was part of the symposium, UK a

In [14]:
df['text'].value_counts()[df['text'].value_counts()>3]

Economics\n\nBetsey Stevenson, professor of public policy and economics at University of Michigan, leads a conversation on the new U.S. stimulus bill and how it will shape U.S. economic competitiveness.\n\nVirtual Event with Betsey Stevenson State and Local Conference Calls and Webinars    2353
                                                                                                                                                                                                                                                                                                     10
Name: text, dtype: int64

In [15]:
df=df[~df.duplicated('text')]

In [20]:
# df = df[df.text.str.len()>200]
df[df.text.str.len()<300]

,tags,title,date,link,text
3,defense,Reforming the War Powers Resolution for the 21st Century,"March 23, 2021",https://www.cfr.org/report/reforming-war-powers-resolution-21st-century,"Economics\n\nBetsey Stevenson, professor of public policy and economics at University of Michigan, leads a conversation on the new U.S. stimulus bill and how it will shape U.S. economic competitiveness.\n\nVirtual Event with Betsey Stevenson State and Local Conference Calls and Webinars"
69,defense,Global Conflict Tracker,"February 1, 2021",https://www.cfr.org/global-conflict-tracker,The Center for Preventive Action's (CPA) Global Conflict Tracker is an interactive guide to ongoing conflicts around the world of concern to the United States. The interactive covers nearly thirty conflicts with background information and resources on each conflict.


In [25]:
df_conflict = pd.read_csv('cfr_global_conflict.csv', header=0)
df_conflict = df_conflict.rename(columns={'loc':'title'})
df_conflict['tags']='defense'
df_conflict = df_conflict[df_conflict.text.str.len()>200]
df_conflict

,title,link,text,tags
0,War in Afghanistan,https://www.cfr.org/global-conflict-tracker/conflict/war-afghanistan,"Recent Developments\n\nAfter more than a year of direct negotiations, the U.S. government and the Taliban signed a peace agreement on February 29, 2020, that sets a timeline for the withdrawal of U.S. forces from Afghanistan. Under the agreement, the United States will draw down U.S. forces to approximately 8,500 troops within 135 days and complete a full withdrawal within fourteen months. In return, the Taliban pledged to prevent territory under their control from being used by terrorist gr...",defense
1,Civil War in Syria,https://www.cfr.org/global-conflict-tracker/conflict/civil-war-syria,"Recent Developments\n\nIn December 2018, President Donald J. Trump announced a decision to withdraw the roughly two thousand U.S. troops remaining in Syria. On January 16, 2019, an attack in Manbij claimed by the self-proclaimed Islamic State killed at least nineteen people, including four Americans. Prior to that attack, only two Americans had been killed in action in Syria since the U.S.-led campaign began. The U.S.-led international coalition continues to carry out military strikes agains...",defense
2,Territorial Disputes in the South China Sea,https://www.cfr.org/global-conflict-tracker/conflict/territorial-disputes-south-china-sea,"Recent Developments\n\nTensions between China and both the Philippines and Vietnam have recently cooled, even as China increased its military activity in the South China Sea by conducting a series of naval maneuvers and exercises in March and April 2018. Meanwhile, China continues to construct military and industrial outposts on artificial islands it has built in disputed waters.\n\nThe United States has also stepped up its military activity and naval presence in the region in recent years, ...",defense
3,Tensions in the East China Sea,https://www.cfr.org/global-conflict-tracker/conflict/tensions-east-china-sea,"Recent Developments\n\nTensions between China and Japan over the contested Senkaku/Diaoyu islands continue to increase as both countries improve their military capabilities, particularly their radar and missile systems, in the region. To avoid accidental clashes at air and sea, China and Japan announced a new crisis communication hotline in June 2018. However, although Japan's Ministry of Defense reported that the number of times Japan's military had to scramble jets in response to Chinese a...",defense
4,North Korea Crisis,https://www.cfr.org/global-conflict-tracker/conflict/north-korea-crisis,"Recent Developments\n\nIn early 2018, North and South Korea began a diplomatic rapprochement, and North Korean officials attended the Opening Ceremony of the Winter Olympics in Pyeongchang, South Korea. On April 27, a week after announcing that North Korea would freeze weapons and missile testing, North Korean Supreme Leader Kim Jong-un stepped across the border into South Korea for a summit with South Korean President Moon Jae-in. The leaders signed a joint statement pledging to work toward...",defense
5,Political Instability in Iraq,https://www.cfr.org/global-conflict-tracker/conflict/political-instability-iraq,"Recent Developments\n\nIn late April 2018, the U.S. military officially disbanded the command overseeing the fight against the self-proclaimed Islamic State in Iraq, declaring an end to major combat operations against the group. More than five thousand U.S. service members remain in Iraq as part of a train, advise, and assist mission bolstered by NATO troops, to help train the Iraqi military and stabilize the country.\n\nA coalition of parties led by Shiite cleric Muqtada al-Sadr won a surpr...",defense
6,Islamist Militancy in Pakistan,https://www.cfr.org/global-conflict-tracker/conflict/islamist-militancy-pakistan,"Recent Developments\n\nPakistan continues to face significant threats to its internal security from factions of the Tehrik-e-Taliban Pakistan (TTP) and other militant group

In [ ]:
df.shape, df_conflict.shape

In [27]:
df = pd.concat([df, df_conflict]).reset_index(drop=True)

In [31]:
df= df.rename(columns={'tags':'topic','link':'url'})
df

,topic,title,date,url,text
0,defense,"Cyber Week in Review: March 26, 2021","March 26, 2021",https://www.cfr.org/blog/cyber-week-review-march-26-2021,"China Restricts Use of Tesla Vehicles by Military and State Employees\n\nFollowing a government security review, the Chinese military has announced that it will restrict the use of Tesla Inc’s vehicles by both military staff and employees of certain state-owned enterprises. The ban is in response to security concerns regarding Tesla’s ubiquitous camera and telecommunications technology, which Beijing believes could facilitate covert data extraction and surveillance. The move mirrors U.S. res..."
1,defense,Armenia’s Postwar Crisis: What to Know,"March 25, 2021",https://www.cfr.org/in-brief/armenias-postwar-crisis-what-know,"Armenian Prime Minister Nikol Pashinyan has called for early elections after weeks of protests and a standoff with his own army. What’s going on?\n\nPashinyan, Armenia’s forty-five-year-old prime minister who was elected in 2018 on the back of a so-called Velvet Revolution, is banking on early elections set for June 20 to quell a political crisis that has its roots in Armenia’s bitter defeat in a forty-four-day war with neighboring Azerbaijan last year.\n\nPashinyan himself triggered the cri..."
2,defense,The Public Should Be Warned When a Rampage Is Underway,"March 24, 2021",https://www.cfr.org/article/public-should-be-warned-when-rampage-underway,"Mass shootings and domestic terrorism are often remembered in terms of places and buildings—single symbolic locations where horror unfolded. The historic Emanuel African Methodist Episcopal Church in Charleston, South Carolina; a Walmart in El Paso, Texas; and the Tree of Life Congregation synagogue in Pittsburgh all readily come to mind.\n\nBut many assailants embark on paths of destruction across multiple crime scenes. An attack that begins as what seems to be an isolated incident at one s..."
3,defense,Reforming the War Powers Resolution for the 21st Century,"March 23, 2021",https://www.cfr.org/report/reforming-war-powers-resolution-21st-century,"Economics\n\nBetsey Stevenson, professor of public policy and economics at University of Michigan, leads a conversation on the new U.S. stimulus bill and how it will shape U.S. economic competitiveness.\n\nVirtual Event with Betsey Stevenson State and Local Conference Calls and Webinars"
4,defense,"Multiple Jihadi Insurgencies, Cooperating With Bandits, Appear to be Converging in the Sahel","March 23, 2021",https://www.cfr.org/blog/multiple-jihadi-insurgencies-cooperating-bandits-appear-be-converging-sahel,"On March 17, around one hundred assailants, traveling on motorcycles and pickup trucks, killed thirty-three soldiers and wounded an additional fourteen in an attack near Tessit in central Mali. Peacekeepers operating under the UN Multidimensional Integrated Stabilization Mission in Mali (MINUSMA) assisted the evacuation of the dead and wounded; the French-led counterinsurgency force stationed in West Africa, Operation Barkhane, helped the Malian military secure the area after the attack. Abo..."
...,...,...,...,...,...
12526,defense,Instability in Venezuela,NaN,https://www.cfr.org/global-conflict-tracker/conflict/instability-venezuela,"Recent Developments\n\nVenezuela is in the midst of a humanitarian crisis. Thousands of people flee the country every day, mostly on foot. In April 2019, after years of denying the existence of a humanitarian crisis and refusing to allow foreign aid to enter the country—calling aid shipments a political ploy by the United States—Venezuelan President Nicolás Maduro allowed the entry of a shipment of emergency supplies from the Red Cross. Venezuela’s infrastructure has been poorly maintained, ..."
12527,defense,Violence in the Democratic Republic of Congo,NaN,https://www.cfr.org/global-conflict-tracker/conflict/violence-democratic-republic-congo,"Recent Developments\n\nOpposition leader Félix Tshisekedi was declared the winner of the Dem

In [33]:
df = df.sort_values('date')
df

,topic,title,date,url,text
6398,diplomacy,Financing America's Leadership,"April 1, 1997",https://www.cfr.org/report/financing-americas-leadership,"Relative to the average of the 1980's, U.S. government spending on international affairs has fallen nearly 20 percent in real terms and will decline by as much as another 30 percent under budget-balancing plans proposed by the president and Congress.\n\nAn Independent Task Force convened by the Brookings Institution and the Council on Foreign Relations was asked to assess the consequences of this trend and to make appropriate recommendations. In its Statement, the Task Force concludes that t..."
9485,politics,U.S. Policy Toward Northeastern Europe,"April 1, 1999",https://www.cfr.org/report/us-policy-toward-northeastern-europe,"Task Force Members\n\nTask Force Members\n\nANDERS ASLUND is senior associate at the Carnegie Endowment for International Peace in Washington, DC.\n\nIAN J. BRZEZINSKI is legislative assistant for national security affairs to Senator William V. Roth Jr. (R-Del.).\n\nARIEL COHEN is senior policy analyst at the Heritage Foundation in Washington, DC.\n\nJOSEPH J. COLLINS is senior fellow in political-military studies at the Center for Strategic and International Studies in Washington, DC. He is..."
3476,defense,"China, Nuclear Weapons, and Arms Control","April 1, 2000",https://www.cfr.org/report/china-nuclear-weapons-and-arms-control,"Few challenges loom as large on the U.S. foreign policy agenda as the effective management of relations with the People's Republic of China (PRC). This is a perennial challenge, given China's central role in Asia and the many issues on the bilateral agenda that feature prominently in U.S. domestic politics. But U.S.-PRC relations take on added significance with China's emergence after decades of isolation and its growing weight in the global economy. Cooperation between China and the United ..."
6382,diplomacy,The Collapse of the Kyoto Protocol and the Struggle to Slow Global Warming,"April 1, 2001",https://www.cfr.org/book/collapse-kyoto-protocol-and-struggle-slow-global-warming,"Most politicians, policy-makers, and analysts hailed the Kyoto Protocol as a vital first step toward slowing greenhouse warming. Council Senior Fellow David Victor was not among them. In this clear and cogent book, Victor explains why the Kyoto Protocol is unlikely to enter into force and how its failure will offer the opportunity to establish a more realistic alternative.\n\nKyoto's fatal flaw, Victor argues, is that it can work only if so-called emissions trading works. The protocol requir..."
9469,politics,Council Expert Says State Department and Pentagon at Odds Over Postwar Iraq Policy; Asserts,"April 1, 2003",https://www.cfr.org/interview/council-expert-says-state-department-and-pentagon-odds-over-postwar-iraq-policy-asserts,"Arthur C. Helton, the Council on Foreign Relations humanitarian affairs expert, warns that the Bush administrations go-it-alone policy has caused a crisis of legitimacy that has blocked many potential donors from participating in post-war reconstruction in Iraq. He also says that divisions between the State Department and the Pentagon on who should run post-war aid efforts have helped fragment planning.\n\nNo one country, even one as powerful as the United States, can hope to do all tha..."
...,...,...,...,...,...
12526,defense,Instability in Venezuela,NaN,https://www.cfr.org/global-conflict-tracker/conflict/instability-venezuela,"Recent Developments\n\nVenezuela is in the midst of a humanitarian crisis. Thousands of people flee the country every day, mostly on foot. In April 2019, after years of denying the existence of a humanitarian crisis and refusing to allow foreign aid to enter the country—calling aid shipments a political ploy by the United States—Venezuelan President Nicolás Maduro allowed the entry of a shipment of emergency supplies from the Red Cross. Venezuela’s infrastructure has been poorly maintained, ..."
1

In [34]:
df.to_csv('cfr.csv', index=False)

In [28]:
url =  'https://microsites-live-backend.cfr.org/conflict/map/json'
res =requests.get(url)
js = res.json()  
js

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': ['66.181640625', '34.307143856288']},
   'properties': {'title': 'War in Afghanistan',
    'description': '',
    'id': 'conflict-6471',
    'severity': {'us': {'value': 'critical'},
     'usConflictStatus': {'value': 'worsening'}},
    'region': '372',
    'conflictType': '1099',
    'image': 'https://microsites-live-backend.cfr.org/cdn/ff/B6k51l6QgBtSoxsGP89YQ_5n0VyqeqYYWyPWOemMjWM/1590661141/public/styles/700x/public/2019-07/RTR4YQB7%20Afghanistan%20woman%20checkpoint%20army_1.jpg?itok=XrPCzUGv',
    'link': '/global-conflict-tracker/conflict/war-afghanistan'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': ['38.49609375', '35.317366329238']},
   'properties': {'title': 'Civil War in Syria',
    'description': '',
    'id': 'conflict-6472',
    'severity': {'us': {'value': 'significant'},
     'usConflictStatus': {'value': 'unchanging'}},
  

In [36]:
df_conflict = pd.json_normalize(js['features'])[['properties.title','properties.link']]
df_conflict.columns = ['loc','link']
df_conflict.link = 'https://www.cfr.org' + df_conflict.link
df_conflict

,loc,link
0,War in Afghanistan,https://www.cfr.org/global-conflict-tracker/conflict/war-afghanistan
1,Civil War in Syria,https://www.cfr.org/global-conflict-tracker/conflict/civil-war-syria
2,Territorial Disputes in the South China Sea,https://www.cfr.org/global-conflict-tracker/conflict/territorial-disputes-south-china-sea
3,Tensions in the East China Sea,https://www.cfr.org/global-conflict-tracker/conflict/tensions-east-china-sea
4,North Korea Crisis,https://www.cfr.org/global-conflict-tracker/conflict/north-korea-crisis
5,Political Instability in Iraq,https://www.cfr.org/global-conflict-tracker/conflict/political-instability-iraq
6,Islamist Militancy in Pakistan,https://www.cfr.org/global-conflict-tracker/conflict/islamist-militancy-pakistan
7,Political Instability in Lebanon,https://www.cfr.org/global-conflict-tracker/conflict/political-instability-lebanon
8,Instability in Egypt,https://www.cfr.org/global-conflict-tracker/conflict/instability-egypt
9,Conflict in Ukraine,https://www.cfr.org/global-conflict-tracker/conflict/conflict-ukraine


In [47]:
def extract_conflict_text(url):
    try:
        a = Article(url)
        a.download()
        a.parse()
        return a.text
    except:
        return 'error'

In [48]:
pandarallel.initialize(nb_workers=26)
df_conflict[['text']] = df_conflict['link'].parallel_apply(extract_conflict_text).values

INFO: Pandarallel will run on 26 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [49]:
df_conflict = df_conflict[df_conflict.text != 'error'].reset_index(drop=True)
df_conflict

,loc,link,text
0,War in Afghanistan,https://www.cfr.org/global-conflict-tracker/conflict/war-afghanistan,"Recent Developments\n\nAfter more than a year of direct negotiations, the U.S. government and the Taliban signed a peace agreement on February 29, 2020, that sets a timeline for the withdrawal of U.S. forces from Afghanistan. Under the agreement, the United States will draw down U.S. forces to approximately 8,500 troops within 135 days and complete a full withdrawal within fourteen months. In return, the Taliban pledged to prevent territory under their control from being used by terrorist gr..."
1,Civil War in Syria,https://www.cfr.org/global-conflict-tracker/conflict/civil-war-syria,"Recent Developments\n\nIn December 2018, President Donald J. Trump announced a decision to withdraw the roughly two thousand U.S. troops remaining in Syria. On January 16, 2019, an attack in Manbij claimed by the self-proclaimed Islamic State killed at least nineteen people, including four Americans. Prior to that attack, only two Americans had been killed in action in Syria since the U.S.-led campaign began. The U.S.-led international coalition continues to carry out military strikes agains..."
2,Territorial Disputes in the South China Sea,https://www.cfr.org/global-conflict-tracker/conflict/territorial-disputes-south-china-sea,"Recent Developments\n\nTensions between China and both the Philippines and Vietnam have recently cooled, even as China increased its military activity in the South China Sea by conducting a series of naval maneuvers and exercises in March and April 2018. Meanwhile, China continues to construct military and industrial outposts on artificial islands it has built in disputed waters.\n\nThe United States has also stepped up its military activity and naval presence in the region in recent years, ..."
3,Tensions in the East China Sea,https://www.cfr.org/global-conflict-tracker/conflict/tensions-east-china-sea,"Recent Developments\n\nTensions between China and Japan over the contested Senkaku/Diaoyu islands continue to increase as both countries improve their military capabilities, particularly their radar and missile systems, in the region. To avoid accidental clashes at air and sea, China and Japan announced a new crisis communication hotline in June 2018. However, although Japan's Ministry of Defense reported that the number of times Japan's military had to scramble jets in response to Chinese a..."
4,North Korea Crisis,https://www.cfr.org/global-conflict-tracker/conflict/north-korea-crisis,"Recent Developments\n\nIn early 2018, North and South Korea began a diplomatic rapprochement, and North Korean officials attended the Opening Ceremony of the Winter Olympics in Pyeongchang, South Korea. On April 27, a week after announcing that North Korea would freeze weapons and missile testing, North Korean Supreme Leader Kim Jong-un stepped across the border into South Korea for a summit with South Korean President Moon Jae-in. The leaders signed a joint statement pledging to work toward..."
5,Political Instability in Iraq,https://www.cfr.org/global-conflict-tracker/conflict/political-instability-iraq,"Recent Developments\n\nIn late April 2018, the U.S. military officially disbanded the command overseeing the fight against the self-proclaimed Islamic State in Iraq, declaring an end to major combat operations against the group. More than five thousand U.S. service members remain in Iraq as part of a train, advise, and assist mission bolstered by NATO troops, to help train the Iraqi military and stabilize the country.\n\nA coalition of parties led by Shiite cleric Muqtada al-Sadr won a surpr..."
6,Islamist Militancy in Pakistan,https://www.cfr.org/global-conflict-tracker/conflict/islamist-militancy-pakistan,"Recent Developments\n\nPakistan continues to face significant threats to its internal security from factions of the Tehrik-e-Taliban Pakistan (TTP) and other militant groups, including an affiliate of the self-proclaimed Islami

In [54]:
df_conflict.to_csv('cfr_global_conflict.csv', index=False)
df_conflict.text.to_csv('cfr_global_conflict_only_text.csv', index=False)